In [ ]:
import ML4PS as ml
import numpy as np
from matplotlib import pyplot as plt

# Loading and preprocessing the data

In [ ]:
data_dir = '../../data/case14'

normalizer = ml.Normalizer(data_dir = data_dir, backend_name = 'pandapower')

interface = ml.Interface(data_dir = data_dir,
    backend_name = 'pandapower', batch_size = 1)

# Defining a Hyper Heterogeneous Multi Graph Neural Ordinary Differential Equation (H2MGNODE)
The H2MGNODE combines the Hyper Heterogeneous Multi Graph Neural Network and Neural Ordinary Differential Equations.

It uses an architecture very similar to the H2MGNN architecture, but it solves the latent system of equations using a variable timestep method. Contrarily to the fully connected, you do not have to specify the amount of objects of each class at the initialization. It will take as input both the addresses $a$ and the features $x$. One should tell the H2MGNODE which addresses it should be using to propagate information.

In [ ]:
addresses = {}
addresses['bus']       = ['id']
addresses['gen']       = ['bus']
addresses['load']      = ['bus']
addresses['shunt']     = ['bus']
addresses['ext_grid']  = ['bus']
addresses['line']      = ['from_bus', 'to_bus']
addresses['trafo']     = ['hv_bus', 'lv_bus']
addresses['poly_cost'] = ['element']

input_features = {}
input_features['bus']       = ['in_service', 'max_vm_pu', 'min_vm_pu', 'vn_kv']
input_features['load']      = ['const_i_percent', 'const_z_percent', 'controllable', 'in_service', 
                               'p_mw', 'q_mvar', 'scaling', 'sn_mva']
input_features['gen']       = ['in_service', 'p_mw', 'scaling', 'sn_mva', 'vm_pu', 'slack', 'max_p_mw', 
                               'min_p_mw', 'max_q_mvar', 'min_q_mvar', 'slack_weight']
input_features['shunt']     = ['q_mvar', 'p_mw', 'vn_kv', 'step', 'max_step', 'in_service']
input_features['ext_grid']  = ['in_service', 'va_degree', 'vm_pu', 'max_p_mw', 'min_p_mw', 'max_q_mvar',
                               'min_q_mvar', 'slack_weight']
input_features['line']      = ['c_nf_per_km', 'df', 'g_us_per_km', 'in_service', 'length_km', 'max_i_ka',
                               'max_loading_percent', 'parallel', 'r_ohm_per_km', 'x_ohm_per_km']
input_features['trafo']     = ['df', 'i0_percent', 'in_service', 'max_loading_percent', 'parallel', 
                               'pfe_kw', 'shift_degree', 'sn_mva', 'tap_max', 'tap_neutral', 'tap_min', 
                               'tap_phase_shifter', 'tap_pos', 'tap_side', 'tap_step_degree', 
                               'tap_step_percent', 'vn_hv_kv', 'vn_lv_kv', 'vk_percent', 'vkr_percent']
input_features['poly_cost'] = ['cp0_eur', 'cp1_eur_per_mw', 'cp2_eur_per_mw2', 'cq0_eur', 
                               'cq1_eur_per_mvar', 'cq2_eur_per_mvar2']

output_features = {}
output_features['bus'] = ['res_vm_pu']

In [ ]:
h2mgnode = ml.H2MGNODE(addresses=addresses, 
                       input_features = input_features, 
                       output_features = output_features, 
                       hidden_dimensions = [64], 
                       latent_dimension = 32)

In [ ]:
class PostProcessor:
    def __call__(self, y):
        return {'bus': {'res_vm_pu': self.bus_v_mag(y['bus']['res_vm_pu'])}}
    def bus_v_mag(self, y):
        return 1.+y
postprocessor = PostProcessor()

In [ ]:
def loss(params, init_state, y_truth):
    y_hat = h2mgnode.solve_and_decode_batch(params, init_state)
    y_post = postprocessor(y_hat)
    return ml.mean((y_truth['bus']['res_vm_pu'] - y_post['bus']['res_vm_pu'])**2)

@ml.jit
def update(params, init_state, y_truth, opt_state):
    value, grads = ml.value_and_grad(loss)(params, init_state, y_truth)
    opt_state = opt_update(0, grads, opt_state)
    return get_params(opt_state), opt_state, value

In [ ]:
step_size = 3e-4
opt_init, opt_update, get_params = ml.optimizers.adam(step_size)
opt_state = opt_init(h2mgnode.weights)

In [ ]:
for epoch in range(5):
    
    # Training loop
    train_losses = []
    for a, x, nets in interface.get_train_batch():
        interface.run_load_flow_batch(nets)
        y_truth = interface.get_features_dict(nets, {'bus':['res_vm_pu']})
        
        x_norm = normalizer(x)
        init_state = h2mgnode.init_state_batch(a, x_norm)
        h2mgnode.weights, opt_state, train_loss = update(h2mgnode.weights,
            init_state, y_truth, opt_state)

        train_losses.append(train_loss)
    
    # Compute metrics over validation set
    val_losses = []
    for a, x, nets in interface.get_val_batch():
        interface.run_load_flow_batch(nets)
        y_truth = interface.get_features_dict(nets, {'bus':['res_vm_pu']})
    
        x_norm = normalizer(x)
        init_state = h2mgnode.init_state_batch(a, x_norm)
        val_loss = loss(h2mgnode.weights, init_state, y_truth)
        val_losses.append(val_loss)
        
    print("Epoch {}".format(epoch))
    print("    Train mean loss = {:.2e}".format(np.mean(train_losses)))
    print("    Validation mean loss = {:.2e}".format(np.mean(val_losses)))
    
# saving the trained method
h2mgnode.save('h2mgnode.pkl')